In [ ]:
# pip install llama-index-utils-workflow
# TAVILY_API_KEY, OPENAI_API_KEY

import os
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv())

## 3. Event

In [ ]:
from llama_index.core.workflow import Event
from llama_index.core.schema import Document, NodeWithScore

class SearchEvent(Event):
    """Result of travily search"""

    docs: list[Document]


class CreateCitationsEvent(Event):
    """Add citations to the nodes."""

    nodes: list[NodeWithScore]

## 4. vars

In [ ]:
# CITATION_QA_TEMPLATE, CITATION_REFINE_TEMPLATE, 
# DEFAULT_CITATION_CHUNK_SIZE = 512
# DEFAULT_CITATION_CHUNK_OVERLAP = 20

from llama_index.core.prompts import PromptTemplate

CITATION_QA_TEMPLATE_EN = PromptTemplate(
    "Please provide an answer based solely on the provided sources. "
    "When referencing information from a source, "
    "cite the appropriate source(s) using their corresponding numbers. "
    "Every answer should include at least one source citation. "
    "Only cite a source when you are explicitly referencing it. "
    "If none of the sources are helpful, you should indicate that. "
    "For example:\n"
    "Source 1:\n"
    "The sky is red in the evening and blue in the morning.\n"
    "Source 2:\n"
    "Water is wet when the sky is red.\n"
    "Query: When is water wet?\n"
    "Answer: Water will be wet when the sky is red [2], "
    "which occurs in the evening [1].\n"
    "Now it's your turn. Below are several numbered sources of information:"
    "\n------\n"
    "{context_str}"
    "\n------\n"
    "Query: {query_str}\n"
    "Answer: "
)

CITATION_QA_TEMPLATE = PromptTemplate(
    "請僅根據所提供的來源回答問題。"
    "在引用某個來源的資訊時，"
    "請使用對應的編號來標註來源。"
    "每個答案都必須至少包含一個來源的引用。"  # 答案 / 陳述句
    "僅在明確引用該來源時才標註來源。"
    "如果沒有任何來源有幫助，你應該指出這一點。"  # 所以其實自帶了 filter
    "例如：\n"
    "來源 1:\n"
    "如果一隻土撥鼠會丟木頭，那牠能丟的木頭量，就等於「一隻會丟木頭的土撥鼠所能丟的木頭量」。\n"
    "來源 2:\n"
    "哪有土撥鼠真的會丟木頭。\n"
    "問題：一隻土撥鼠如果會丟木頭，那牠能丟多少木頭？\n"
    "答案：一般來說，土撥鼠其實並不會真的丟木頭 [2]。"
    "不過如果牠真的會丟木頭，那牠能丟的數量，就等於一隻會丟木頭的土撥鼠所能丟的數量 [1]。\n"
    "現在輪到你了。以下是數個已編號的資訊來源："
    "\n------\n"
    "{context_str}"
    "\n------\n"
    "問題：{query_str}\n"
    "答案："
)

CITATION_REFINE_TEMPLATE_EN = PromptTemplate(
    "Please provide an answer based solely on the provided sources. "
    "When referencing information from a source, "
    "cite the appropriate source(s) using their corresponding numbers. "
    "Every answer should include at least one source citation. "
    "Only cite a source when you are explicitly referencing it. "
    "If none of the sources are helpful, you should indicate that. "
    "For example:\n"
    "Source 1:\n"
    "The sky is red in the evening and blue in the morning.\n"
    "Source 2:\n"
    "Water is wet when the sky is red.\n"
    "Query: When is water wet?\n"
    "Answer: Water will be wet when the sky is red [2], "
    "which occurs in the evening [1].\n"
    "Now it's your turn. "
    "We have provided an existing answer: {existing_answer}"
    "Below are several numbered sources of information. "
    "Use them to refine the existing answer. "
    "If the provided sources are not helpful, you will repeat the existing answer."
    "\nBegin refining!"
    "\n------\n"
    "{context_msg}"
    "\n------\n"
    "Query: {query_str}\n"
    "Answer: "
)


CITATION_REFINE_TEMPLATE = PromptTemplate(
    "請僅根據所提供的來源來生成答案。"
    "在引用來源中的資訊時，"
    "請使用對應的來源編號進行標註。"
    "每個答案都必須至少包含一個來源引用。"
    "僅在你明確參考來源時才進行引用。"
    "如果提供的來源沒有幫助，你應該直接重複既有的答案。"
    "例如：\n"
    "來源 1：\n"
    "如果一隻土撥鼠會丟木頭，那牠能丟的木頭量，就等於「一隻會丟木頭的土撥鼠所能丟的木頭量」。\n"
    "來源 2：\n"
    "哪有土撥鼠真的會丟木頭。\n"
    "問題：一隻土撥鼠如果會丟木頭，那牠能丟多少木頭？\n"
    "答案：一般來說，土撥鼠其實並不會真的丟木頭 [2]。"
    "不過如果牠真的會丟木頭，那牠能丟的數量，就等於一隻會丟木頭的土撥鼠所能丟的數量 [1]。\n"
    "現在輪到你了。"
    "這裡有一個既有答案：{existing_answer}\n"
    "以下是幾個編號的資訊來源。"
    "請使用這些來源來改進既有的答案。"
    "如果來源沒有幫助，你就直接重複既有答案。"
    "\n開始改進！"
    "\n------\n"
    "{context_msg}"
    "\n------\n"
    "問題：{query_str}\n"
    "答案："
)

DEFAULT_CITATION_CHUNK_SIZE = 512
DEFAULT_CITATION_CHUNK_OVERLAP = 20

## 4. Defining the workflow

In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.workflow import (
    Context,
    Workflow,
    StartEvent,
    StopEvent,
    step,
)

from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

from llama_index.core.schema import (
    MetadataMode,
    NodeWithScore,
    TextNode,
)

from llama_index.core.response_synthesizers import (
    ResponseMode,
    get_response_synthesizer,
)

from typing import Union, List
from llama_index.core.node_parser import SentenceSplitter

from llama_index.tools.tavily_research.base import TavilyToolSpec


TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
tavily_tool = TavilyToolSpec(
    api_key=TAVILY_API_KEY,
)

In [ ]:
class CitationQueryEngineWorkflow(Workflow):
    @step
    async def search(
        self, ctx: Context, ev: StartEvent
    ) -> Union[SearchEvent, None]:
        "Entry point for RAG, triggered by a StartEvent with `query`."
        query = ev.get("query")
        if not query:
            return None

        print(f"Query the tavily with: {query}")

        # store the query in the global context
        await ctx.store.set("query", query)

        docs = tavily_tool.search(query, max_results=3)
        print(f"get {len(docs)} docs.")
        return SearchEvent(docs=docs)

    @step
    async def create_citation_nodes(
        self, ev: SearchEvent
    ) -> CreateCitationsEvent:
        """
        Returns:
            List[NodeWithScore]: A list of NodeWithScore objects, where each object
            represents a smaller chunk of the original docs, labeled as a source.
        """
        docs = ev.docs

        nodes: List[NodeWithScore] = []

        text_splitter = SentenceSplitter(
            chunk_size=DEFAULT_CITATION_CHUNK_SIZE,
            chunk_overlap=DEFAULT_CITATION_CHUNK_OVERLAP,
        )

        for doc in docs:
            text_chunks = text_splitter.split_text(
                doc.get_content(metadata_mode=MetadataMode.NONE)
            )

            for text_chunk in text_chunks:
                text = f"Source {len(nodes)+1}:\n{text_chunk}\n"
                
                text_node = TextNode(
                    text=text,
                    metadata=doc.metadata,
                    id_=doc.doc_id  # 注意：Document 有 doc_id 屬性
                )

                node = NodeWithScore(node=text_node, score=1.0)
                nodes.append(node)
        return CreateCitationsEvent(nodes=nodes)

    @step
    async def synthesize(
        self, ctx: Context, ev: CreateCitationsEvent
    ) -> StopEvent:
        """Return a streaming response using the retrieved nodes."""
        llm = OpenAI(model="gpt-5-mini")
        query = await ctx.store.get("query", default=None)

        synthesizer = get_response_synthesizer(
            llm=llm,
            text_qa_template=CITATION_QA_TEMPLATE,
            refine_template=CITATION_REFINE_TEMPLATE,
            response_mode=ResponseMode.COMPACT,
            use_async=True,
        )

        response = await synthesizer.asynthesize(query, nodes=ev.nodes)
        return StopEvent(result=response)

In [ ]:
from llama_index.utils.workflow import draw_all_possible_flows

draw_all_possible_flows(
    CitationQueryEngineWorkflow,
    filename="Search_CitationQueryEngine_Workflow.html",
    # Optional, can limit long event names in your workflow
    # Can help with readability
    # max_label_length=10,
)

In [ ]:
w = CitationQueryEngineWorkflow()
# Run a query
query = '徵象(Signs)及症狀(Symptoms)之區別?'
result = await w.run(query=query)
print(result.response)

In [ ]:
result.response

In [ ]:
print(result.source_nodes[0].node.get_text())

In [ ]:
print(result.source_nodes[2].node.get_text())